In [1]:
# i want to finetune a model to do credit card fraud detection. A sample input is given below.

# the huggingface dataset for legitimate transactions is called "LouisXO/fraud-detection-legitimate"

# the huggingface dataset for fraudulent transactions is called "LouisXO/fraud-detection-all-fraud"

# all datasets have columns "conversation" and "response" 
# the response is either "LEGITIMATE" or "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: fraud_Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR

# response: LEGITIMATE



In [2]:
# Import libraries
import datasets
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import wandb  # Add this line

wandb.init(project="fraud_detection")

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aidenyang66 (yyfsss). Use `wandb login --relogin` to force relogin


In [23]:
# Load legitimate transactions dataset
legitimate_dataset = load_dataset("LouisXO/fraud-detection-legitimate")

# Load fraudulent transactions dataset
fraudulent_dataset = load_dataset("LouisXO/fraud-detection-all-fraud")

In [24]:
# Assign label 0 to legitimate transactions
legitimate_dataset = legitimate_dataset.map(lambda x: {'label': 0})

# Assign label 1 to fraudulent transactions
fraudulent_dataset = fraudulent_dataset.map(lambda x: {'label': 1})

# Combine the datasets
full_dataset = concatenate_datasets([legitimate_dataset['train'], fraudulent_dataset['train']])

In [25]:
# Shuffle the combined dataset
full_dataset = full_dataset.shuffle(seed=42)

# Split into training and validation sets (e.g., 90% train, 10% validation)
split_dataset = full_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [26]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(example):
    return tokenizer(example['conversation'], padding='max_length', truncation=True)

# Apply the tokenizer to the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [27]:
# Load a pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [29]:
training_args = TrainingArguments(
    output_dir='./results',            # Output directory
    evaluation_strategy='steps',       # Evaluate every N steps
    save_strategy='steps',             # Save the model every N steps
    eval_steps=500,                    # Evaluation interval
    save_steps=500,                    # Save interval
    num_train_epochs=1,                # Number of training epochs
    per_device_train_batch_size=8,    # Batch size for training
    per_device_eval_batch_size=8,     # Batch size for evaluation
    logging_dir='./logs',              # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,       # Load the best model when finished training
    report_to="wandb",                 
    
)

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\transformers\training_args.py:1541: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  
)

# Train the model
trainer.train()

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\wandb\sdk\wandb_run.py:2382: UserWarning: Run (uump7kna) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
  0%|          | 10/12095 [00:01<36:23,  5.53it/s]
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\wandb\sdk\wandb_run.py:2382: UserWarning: Run (07utm1ql) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
                                                

                                                  
  0%|          | 11/12095 [00:01<36:44,  5.48it/s]

{'loss': 0.4332, 'grad_norm': 2.2821335792541504, 'learning_rate': 4.9958660603555194e-05, 'epoch': 0.0}


  0%|          | 20/12095 [00:03<36:20,  5.54it/s]
                                                

                                                  
  0%|          | 21/12095 [00:03<36:45,  5.48it/s]

{'loss': 0.312, 'grad_norm': 1.4064308404922485, 'learning_rate': 4.991732120711038e-05, 'epoch': 0.0}


  0%|          | 30/12095 [00:05<36:49,  5.46it/s]
                                                

                                                  
  0%|          | 31/12095 [00:05<37:15,  5.40it/s]

...

  4%|▍         | 490/12095 [01:30<35:29,  5.45it/s]
                                                

                                                   
  4%|▍         | 491/12095 [01:30<35:52,  5.39it/s]

{'loss': 0.0107, 'grad_norm': 0.2641292214393616, 'learning_rate': 4.797436957420422e-05, 'epoch': 0.04}


  4%|▍         | 500/12095 [01:32<35:19,  5.47it/s]
                                                

                                                   
  4%|▍         | 500/12095 [01:32<35:19,  5.47it/s]

{'loss': 0.1939, 'grad_norm': 0.19023583829402924, 'learning_rate': 4.793303017775941e-05, 'epoch': 0.04}


KeyboardInterrupt: 

In [ ]:
# Evaluate the model
evaluation_results = trainer.evaluate()

# Print evaluation results
print(evaluation_results)